# Sage Special Topics: Cython

## Overview

[Cython](https://cython.org/) is a compiler  
for the Python programming language along with  
the extended Cython programming language.  
This combination allows you to enjoy  
the readability of Python and 
the efficiency of C.

This means you may:  
- use the **static data type in C** to speed up the program,  
- **wrap C functions** into Python interface, and  
- yet more features that I don't totally understand.

Sage developing team plays an important role  
in building up Cython; see [the history of Cython](https://en.wikipedia.org/wiki/Cython#History).  
However, Cython is not limited to the Sage environment,  
and this tutorial is also applicable for general Python users.

## Installation

If you are using a Sage notebook,  
or a Jupyter notebook under the Sage kernel,  
you may ignore this section.

For general Python users,  
you may use  
```Python
pip install Cython --user
```
to **install** Cython on your machine.  

To use it in a Jupyter notebook under the Python kernel,  
you have to run the following cell  
to **activate** the Cython extension.

In [6]:
%load_ext Cython

For using Cython in the console or other IDEs,  
please refer to the [Workflow](#Workflow) section.

## Workflow

As mentioned before,  
Cython is a **compiler** rather than an interpreter,  
so each piece of the code need to be compiled before execution.

Here is the basic workflow: (See [Basic Tutorial](https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html) of the official Cython documentation for more details.)  
- Prepare a `cython_code.pyx` file that contains your Cython code.
- Prepare a `setup.py` file that contains the setup information.
- run `python setup.py build_ext --inplace` in the terminal  
to generate `cython_code.so` (unix) or `cython_code.pyd` (Windows).
- In another Python file or in a Jupyter notebook, `import cython_code`.

Sample `cython_code.pyx`:

```Python
def print_interest():
    print("I love Math!")
```

Sample `setup.py`:

```Python
from setuptools import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("cython_code.pyx")
)
```

Sample usage:

```Python
import cython_code

print_interest()
```

Jupyter wrap this process  
by the `%%cython` magic function  
(`%cython` in SageNB).  

Let's look at an example below.

## A quick example

Here is a piece of Python code  
that calculates all primes below or equal to `n`.

In [3]:
def a_prime(p):
    """Tell p is a prime or not"""
    for i in range(2,p):
        if p % i == 0:
            return False
    else:
        return True

def primes_below(n):
    """Return all primes below or equal to n"""
    primes = [p for p in range(2,n+1) if a_prime(p)]
    return primes

In [4]:
%%timeit
primes = primes_below(10000)

1 loop, best of 3: 230 ms per loop


If you are using Jupyter,  
simply **add `%%cython` to the first line** of the target cell.  
Thus, the code in the cell will be compiled  
and then imported into the notebook automatically.  
(The same process as in the workflow.)

If you use pure Python,  
remember to activate the extension first.
```Python
%load_ext Cython
```

If you use SageNB, 
then add `%cython` instead.

In [5]:
%%cython

def a_prime_pinc(p):
    """Tell p is a prime or not"""
    for i in range(2,p):
        if p % i == 0:
            return False
    else:
        return True

def primes_below_pinc(n):
    """Return all primes below or equal to n"""
    primes = [p for p in range(2,n+1) if a_prime_pinc(p)]
    return primes

In [6]:
%%timeit
primes = primes_below_pinc(10000)

10 loops, best of 3: 148 ms per loop


By compiling the code through Cython,  
it already speed up the code  
without doing anything further.

If we appropriately change some  
dynamic data types in Python  
to **static data types** in C,  
then this will enhance the performance further.

In [119]:
%%cython

cdef bint a_prime_c(int p):
    cdef int i
    for i in range(2,p):
        if p % i == 0:
            return False
    else:
        return True

cpdef primes_below_c(int n):
    primes = [p for p in range(2,n+1) if a_prime_c(p)]
    return primes

In [120]:
%%timeit
primes = primes_below_c(10000)

100 loops, best of 5: 10.2 ms per loop


## Static data types

Use `cdef` to **declare a static data type** or  
define a function.

In [27]:
%%cython

cdef int a = 1
print(a)

In [32]:
%%cython

cdef int plus_one(int x):
    return x+1

print(plus_one(1))

2


But then this variable or function is not accessible by other cells.

In [ ]:
a ### result in an error

In [ ]:
plus_one(1) ### result in an error

If necessary, use `cpdef` to  
define a function and  
**wrap it with a Python interface**.  

In [37]:
%%cython

cpdef int plus_two(int x):
    return x+2

print(plus_two(1))

3


In [38]:
plus_two(1)

3

Recall that you can still use `def` in Cython,  
but it does not support static typing for the function.  
(Static typing for the arguments is okay.)

In [40]:
%%cython

def plus_three(int x):
    return x+3

print(plus_three(1))

4


In [41]:
plus_three(1)

4

There are many combinations of  
`def`, `cdef`, `cpdef`, with/without static typing.  

Here is an article "[How Fast are def cdef cpdef?](https://notes-on-cython.readthedocs.io/en/latest/fibo_speed.html)"  
that gives expriments on the speed of these combinations.

Possible static data types can be found [here](https://cython.readthedocs.io/en/latest/src/userguide/language_basics.html).

## Communication between Python and Cython

As mentioned, Cython is a compiler  
and the `%%cython` magic function  
automate the process of  
- **compiling** the cell, and  
- **import** the compiled file.

Therefore, variables or functions defined in other cells  
are not known to the cell to be compiled.

In [7]:
a = 1

In [ ]:
%%cython
print(a) ### result in error

After a cell is compiled,  
not all variables or functions in the compiled function  
are known to other cells.

In [11]:
%%cython
b = 1

In [12]:
print(b)

1


In [16]:
%%cython
cdef int c = 1

In [ ]:
print(c) ### result in error

The Cython cell is compiled  
and the `[name].so` or `[name].pyd` file is stored in a temporary directory.  

Then **every Python object** in this `[name].so` or `[name].pyd` file  
is imported, which is similar to `from [name] import *`.

**C variables or C objects will only work when compiling**.  

In [21]:
%%cython

cpdef int f(int x):
    return x + 1

In [18]:
cython??

In [22]:
%%cython
cpdef int a=1

In [23]:
a

1

annotate

In [141]:
%%cython
# distutils: language=c++

from libcpp.vector cimport vector

def cprimes(unsigned int nb_primes):
    cdef int n, i
    cdef vector[int] p
    p.reserve(nb_primes)  # allocate memory for 'nb_primes' elements.

    n = 2
    while p.size() < nb_primes:  # size() for vectors is similar to len()
        for i in p:
            if n % i == 0:
                break
        else:
            p.push_back(n)  # push_back is similar to append()
        n += 1

    # Vectors are automatically converted to Python
    # lists when converted to Python objects.
    return p

# print(cprimes(1229))

In [142]:
%%timeit
cprimes(1229)

100 loops, best of 5: 2.11 ms per loop
